In [1]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [2]:
class SyntheticRegressionData(d2l.DataModule):  #@save
    """Synthetic data for linear regression."""
    def __init__(self, w, b, noise=0.01, num_train=1000, num_val=1000,
                 batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise

In [3]:
data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)

In [4]:
print('features:', data.X[0],'\nlabel:', data.y[0])

features: tensor([ 0.7989, -0.4564]) 
label: tensor([7.3393])


In [5]:
@d2l.add_to_class(SyntheticRegressionData)
def get_dataloader(self, train):
    if train:
        indices = list(range(0, self.num_train))
        # The examples are read in random order
        random.shuffle(indices)
    else:
        indices = list(range(self.num_train, self.num_train+self.num_val))
    for i in range(0, len(indices), self.batch_size):
        batch_indices = torch.tensor(indices[i: i+self.batch_size])
        yield self.X[batch_indices], self.y[batch_indices]

In [6]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


In [7]:
@d2l.add_to_class(d2l.DataModule)  #@save
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    dataset = torch.utils.data.TensorDataset(*tensors)
    return torch.utils.data.DataLoader(dataset, self.batch_size,
                                       shuffle=train)

@d2l.add_to_class(SyntheticRegressionData)  #@save
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)

In [12]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)
print(X,y)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])
tensor([[-0.5038,  0.7829],
        [ 0.1493, -2.4396],
        [ 0.1266,  0.3872],
        [ 0.4122, -0.6263],
        [-1.3054,  0.3417],
        [ 1.1768, -0.1007],
        [-0.5479, -0.2218],
        [-0.9227, -1.6659],
        [-0.6517, -1.5035],
        [-1.5739,  0.1550],
        [-0.0271,  0.3639],
        [-1.6728, -0.5479],
        [ 0.6649, -0.0966],
        [ 0.3601,  0.1022],
        [-2.2594,  0.1973],
        [-0.3508,  0.0817],
        [-1.1934, -0.5339],
        [ 1.3513, -0.9496],
        [ 0.3239,  2.1416],
        [ 0.5009, -0.9392],
        [ 1.3149,  0.7608],
        [-0.3043,  1.0887],
        [ 0.7096, -0.6095],
        [-3.2573, -0.8107],
        [-0.4208,  1.5207],
        [-0.7282,  0.1532],
        [ 0.0148, -1.0422],
        [-0.5293, -1.0436],
        [ 0.3269, -0.0885],
        [-0.0898, -0.9685],
        [ 1.0758, -1.3371],
        [-0.5941, -0.4650]]) tensor([[ 0.5246],
        [12.8132],
      

In [9]:
len(data.train_dataloader())

32